Import Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
#Some configuration for matplotlib and seaborn to look pretty
%matplotlib inline
%config Inlinebackend.figure_format= 'retina'
sns.set_context("poster")
sns.set(rc={"figure.figsize": (18.,9.)})
sns.set_style("whitegrid")
SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 30
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

Read the file and explore the dataset.

In [3]:
df = pd.read_csv('Empresas 2-50M EBITDA.csv')

In [4]:
df.sample()

,Num,Nombre,Código NIF,Localidad,País,Código consolidación,Ultimo año disponible,Ingresos 2021,EBITDA 2021,Código primario CNAE 2009,...,Descripción actividad,Nombre accionista,Accionista - Apellido,Accionista - Nombre,Accionista - Tipo,Accionista - % directo,GUO - Nombre,GUO - Nombre.1,GUO - Ciudad,Category
4766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DSV SOLUTIONS HOLDING A/S,NaN,NaN,Empresa,-,NaN,NaN,NaN,NaN


In [5]:
df.shape

(21452, 22)

The column `Accionista - % directo` comes with values with a "," and "." so we unify them to a "." and reformat other values (e.g. >50.00 --> 50.01)

In [6]:
df['Accionista - % directo'] = df['Accionista - % directo'].apply(lambda x: str(x).replace(',','.') )
df['Accionista - % directo'] = df['Accionista - % directo'].apply(lambda x: x[1:-1]+'1' if '>' in str(x) else x)


Decide which columns have numerical values and remove the text values (e.g.: n.d.)

In [7]:
numeric =  ['Num', 'Ultimo año disponible', 'Ingresos 2021', 'EBITDA 2021', 'Código primario CNAE 2009',
            'EBITDA\nmil EUR\nAño - 1', 'EBITDA\nmil EUR\nAño - 2', 'Accionista - % directo']

In [8]:
for name in df.columns:
    if name in numeric:
        df[name] = pd.to_numeric(df[name], errors='coerce')

In [9]:
df.dtypes

Num                          float64
Nombre                        object
Código NIF                    object
Localidad                     object
País                          object
Código consolidación          object
Ultimo año disponible        float64
Ingresos 2021                float64
EBITDA 2021                  float64
Código primario CNAE 2009    float64
EBITDA\nmil EUR\nAño - 1     float64
EBITDA\nmil EUR\nAño - 2     float64
Descripción actividad         object
Nombre accionista             object
Accionista - Apellido         object
Accionista - Nombre           object
Accionista - Tipo             object
Accionista - % directo       float64
GUO - Nombre                  object
GUO - Nombre.1                object
GUO - Ciudad                  object
Category                     float64
dtype: object

Fill the gaps in column `Nombre` with its corresponding values and check that we didn't miss any.

In [10]:
for i in df.index[1:]:
    if type(df.at[i, 'Nombre']) != str:
        df.at[i, 'Nombre']=df.at[i-1, 'Nombre']

In [11]:
df.isna().sum()

Num                          12753
Nombre                           0
Código NIF                   12753
Localidad                    12753
País                         12753
Código consolidación         12753
Ultimo año disponible        12753
Ingresos 2021                12753
EBITDA 2021                  12753
Código primario CNAE 2009    12753
EBITDA\nmil EUR\nAño - 1     12807
EBITDA\nmil EUR\nAño - 2     12924
Descripción actividad        12771
Nombre accionista                0
Accionista - Apellido        12993
Accionista - Nombre          12993
Accionista - Tipo              240
Accionista - % directo        7184
GUO - Nombre                 14044
GUO - Nombre.1               20069
GUO - Ciudad                 15726
Category                     21452
dtype: int64

Use a category dictionary to simplify column `Accionista - Tipo` to Family, Corporate, Self owned or Not Disclosed. 

In [12]:
dictionary_category = {
    'Una o más personas físicas o familias' : 'Family',
    'Accionistas privados no identificados, agregados' : 'Family',
    'Auto participación' : 'Self owned',
    'Autoridades públicas, Estado, Gobierno' : 'Corporate',
    'Banco' : 'Corporate',
    'Empleados/Administradores/Directores' : 'Family',
    'Empresa' : 'Corporate',
    'Empresa financiera' : 'Corporate',
    'Firmas Private Equity' : 'Corporate',
    'Fondos mutuos & de pensiones/Nominee/Trust/Trustee' : 'Corporate',
    'Fundaciones/Institutos de investigación' : 'Corporate',
    'Otros accionistas no identificados, agregados' : 'Corporate',
    'Público (empresas que cotizan en bolsa)' : 'Corporate',
    'Seguro' : 'Corporate',
    'Una o más personas físicas o familias' : 'Family',
    'Venture capital':'Corporate',
    np.nan : 'Not Disclosed',
}

In [13]:
def define_category(row):
    return dictionary_category[row['Accionista - Tipo']]

In [14]:
df['Accionista - Tipo'] = df.apply(lambda x: define_category(x),axis=1)

Create a new Dataframe `df2` to group companies and its Shareholders' type together

In [15]:
df2 = df.groupby(['Nombre','Accionista - Tipo']).size().reset_index(name='counts')
df3 = df.groupby(['Nombre','Accionista - Tipo']).agg({'Accionista - % directo':sum}).reset_index()
df2['Accionista - % directo'] = df3['Accionista - % directo'] 
df2['Category'] = np.nan
df2.sample()

,Nombre,Accionista - Tipo,counts,Accionista - % directo,Category
9314,TRAJANO IBERIA SOCIMI SA.,Corporate,9,53.9,NaN


Define three lists, one of all company names `companies`, another of companies with a single type of shareholder `pure` and the last one with mixed ownership `mix`

In [16]:
x = pd.DataFrame(df2.Nombre.value_counts()).reset_index()
x = x.rename(columns={"index": "Nombre", "Nombre": "counts"})
companies = x.Nombre.tolist()
pure = x[x.counts==1].Nombre.tolist()
mix = x[x.counts!=1].Nombre.tolist()

In [17]:
print(f'There are a total of {len(companies)} companies, out of which {len(pure)} have a single Shareholder type and {len(mix)} have a mixed ownership.')

There are a total of 8699 companies, out of which 7520 have a single Shareholder type and 1179 have a mixed ownership.


Assign categories for each company, first with the list of `pure` companies

In [18]:
def pure_category(row):
    if row['Nombre'] in pure:
        row['Category'] = row['Accionista - Tipo']
    return row

In [19]:
df = df.apply(lambda x: pure_category(x),axis=1)
df2 = df2.apply(lambda x: pure_category(x),axis=1)

Then we use the list of mixed ownership `mix` and `df2` to determine the kind of ownership these ~1200 companies have.
 - If the total ownership is less than 50% --> Assign `Category` Mix: Other
 - If any grouped ownership is over 50% --> Assign `Category` Mix: Mostly [...] with its corresponding majority.

In [20]:
for name in mix:
    total = sum(df2[df2.Nombre==name]['Accionista - % directo'])
    if total<50:
        df2.loc[df2['Nombre']== name, 'Category'] = 'Mix: Other'

    else:
        category = 'Mix: Other'
        for ownership in df2[df2.Nombre==name]['Accionista - % directo']:
            if ownership>=50:
                category = f"Mix: Mostly {df2.loc[(df2['Nombre']== name)&(df2['Accionista - % directo']== ownership),'Accionista - Tipo'].values[0]}"
        df2.loc[df2['Nombre']== name, 'Category'] = category

Check to see we didn't miss any of the 8699 companies

In [21]:
df2.Category.value_counts()

Corporate                5734
Family                   1534
Mix: Mostly Corporate    1461
Mix: Other                589
Mix: Mostly Family        439
Not Disclosed             240
Self owned                 12
Name: Category, dtype: int64

In [22]:
df2.Category.isna().sum()

0

Transfer the `Category` column from `df2` to `df`

In [23]:
df['Category'] = df.apply(lambda x: df2[df2.Nombre==x.Nombre]['Category'].values[0],axis=1)

Recover the original values for `Accionista - Tipo` in `df`

In [24]:
df['Accionista - Tipo'] = pd.read_csv('Empresas 2-50M EBITDA.csv')['Accionista - Tipo']

Show the % distribution of Type of Ownership from all companies.

In [25]:
round(df2.groupby(['Nombre','Category']).size().reset_index(name='counts')['Category'].value_counts()/len(companies)*100,2)

Corporate                65.92
Family                   17.63
Mix: Mostly Corporate     8.01
Mix: Other                3.17
Not Disclosed             2.76
Mix: Mostly Family        2.37
Self owned                0.14
Name: Category, dtype: float64

Export the results to an excel file

In [26]:
df.to_excel('Companies 2-50M EBITDA.xlsx')